In [1]:
input = "466 players; last marble is worth 71436 points"

"466 players; last marble is worth 71436 points"

In [2]:
test_input = "9 players; last marble is worth 25 points"

"9 players; last marble is worth 25 points"

In [3]:
other_test_inputs = [
    "10 players; last marble is worth 1618 points"
    "13 players; last marble is worth 7999 points"
    "17 players; last marble is worth 1104 points"
    "21 players; last marble is worth 6111 points"
    "30 players; last marble is worth 5807 points"
]

5-element Array{String,1}:
 "10 players; last marble is worth 1618 points"
 "13 players; last marble is worth 7999 points"
 "17 players; last marble is worth 1104 points"
 "21 players; last marble is worth 6111 points"
 "30 players; last marble is worth 5807 points"

In [4]:
parse_input(input) = parse.(Int, match(r"^(\d+).*worth (\d+) points$", input).captures)

parse_input (generic function with 1 method)

In [5]:
move_clockwise(from, steps, circle_length) = mod((from + steps), circle_length)

move_clockwise (generic function with 1 method)

In [6]:
move_counter_clockwise(from, steps, circle_length) = mod((from - steps), circle_length)

move_counter_clockwise (generic function with 1 method)

In [7]:
next_player(player, num_players) = (player) % (num_players) + 1

next_player (generic function with 1 method)

In [8]:
insert_marble!(circle, value, position) = position == 0 ? push!(circle, value) : splice!(circle, position:position-1, value)

insert_marble! (generic function with 1 method)

In [9]:
remove_marble!(circle, position) = splice!(circle, position)

remove_marble! (generic function with 1 method)

In [10]:
function play_game(n_players, n_marbles; debug=false, info=false)
    circle = [0]
    position = 1
    value = 1
    scores = Dict{Int, Int}()
    player = 1
    while value <= n_marbles
        
        if value % 23 == 0
            position = move_counter_clockwise(position, 7, length(circle))
            removed = remove_marble!(circle, position)
            scores[player] = get(scores, player, 0) + value + removed
            !debug ? nothing : println("***Skipped marble $value, removed marble $removed, and set current to $(circle[position])***")
        else
            position = move_clockwise(position, 2, length(circle) + 1)
            insert_marble!(circle, value, position)
            !debug ? nothing : println("Inserted marble $value in $circle, at position $position.")
        end

        !debug ? nothing : println("Circle now has $(length(circle)) marbles:\n$circle\n")
        value += 1
        player = next_player(player, n_players)
    end
    return scores
end

function play_game(input; debug=false)
    n_players, n_marbles = parse_input(input)
    return play_game(n_players, n_marbles, debug=debug)
end

play_game (generic function with 2 methods)

In [11]:
scores = play_game(test_input)
max_score, player = findmax(scores)

(32, 5)

In [12]:
for ti in other_test_inputs
    scores = play_game(ti)
    max_score, player = findmax(scores)
    println("Max Score: $max_score")
end

Max Score: 8317
Max Score: 146373
Max Score: 2764
Max Score: 54718
Max Score: 37305


In [13]:
@time scores = play_game(input, debug=false)
max_score, player = findmax(scores)

  0.191239 seconds (75.69 k allocations: 6.610 MiB)


(382055, 94)

In [14]:
mutable struct Marble
    value::Int
    previous::Marble
    next::Marble
    Marble(value) = (x = new(value); x.previous = x; x.next = x)
    Marble(value, previous, next) = new(value, previous, next)
end

In [15]:
move_clockwise(marble::Marble, steps::Int) = steps == 0 ? marble : move_clockwise(marble.next, steps - 1)

move_clockwise (generic function with 2 methods)

In [16]:
move_counter_clockwise(marble::Marble, steps::Int) = steps == 0 ? marble : move_counter_clockwise(marble.previous, steps - 1)

move_counter_clockwise (generic function with 2 methods)

In [17]:
function insert_marble_before!(marble::Marble, value::Int)
    new_marble = Marble(value, marble.previous, marble)
    marble.previous.next = new_marble
    marble.previous = new_marble
    return new_marble
end

insert_marble_before! (generic function with 1 method)

In [18]:
function remove_marble!(marble::Marble)
    parent = marble.previous
    child = marble.next
    parent.next = child
    child.previous = parent
    return child
end

remove_marble! (generic function with 2 methods)

In [19]:
function show_circle(marble)
    start = marble.value
    circle = [start]
    marble = marble.next
    while marble.value != start
        push!(circle, marble.value)
        marble = marble.next
    end
    return circle
end

show_circle (generic function with 1 method)

In [20]:
function play_game_v2(n_players, n_marbles; debug=false, info=false)
    current = Marble(0)
    root = current

    scores = Dict{Int, Int}()
    value = 1
    player = 1
    while value <= n_marbles
    
        if value % 23 == 0
            removed = move_counter_clockwise(current, 7)
            current = remove_marble!(removed)
            scores[player] = get(scores, player, 0) + value + removed.value
            !debug ? nothing : println("***Skipped marble $value, removed marble $(removed.value), and set current to $(current.value)***")
        else
            mb = move_clockwise(current, 2)
            current = insert_marble_before!(mb, value)
            !debug ? nothing : println("Inserted marble $value just before $(mb.value).")
        end

        if debug
            circle = show_circle(root)
            println("Circle now has $(length(circle)) marbles:\n$circle\n")
        end
        value += 1
        player = next_player(player, n_players)
    end
    return scores
end

function play_game_v2(input; debug=false)
    n_players, n_marbles = parse_input(input)
    return play_game_v2(n_players, n_marbles, debug=debug)
end

play_game_v2 (generic function with 2 methods)

In [21]:
scores = play_game_v2(test_input; debug=false)
max_score, player = findmax(scores)

(32, 5)

In [22]:
for ti in other_test_inputs
    scores = play_game_v2(ti)
    max_score, player = findmax(scores)
    println("Max Score: $max_score")
end

Max Score: 8317
Max Score: 146373
Max Score: 2764
Max Score: 54718
Max Score: 37305


In [23]:
@time scores = play_game_v2(input, debug=false)  # compare speed to v1
max_score, player = findmax(scores)

  0.003114 seconds (68.37 k allocations: 2.110 MiB)


(382055, 94)

In [24]:
n_players, n_marbles = parse_input(input)
@time scores = play_game_v2(n_players, n_marbles * 100, debug=false)
max_score, player = findmax(scores)

  1.361772 seconds (6.83 M allocations: 208.555 MiB, 68.74% gc time)


(3133277384, 9)